In [ ]:
!pip install -r requirements.txt -q

In [ ]:
from helpers.dataset import Conversation
from elasticsearch import Elasticsearch, helpers
from pydantic import BaseModel
import numpy as np
import logging
from typing import List
import os
from helpers.fair_forge import FairForge
import pandas as pd
import math
import re
from collections import defaultdict
from scipy.stats import spearmanr

In [ ]:
ELASTIC_URL = os.environ.get('ELASTIC_URL')
ELASTIC_AUTH = [os.environ.get('ELASTIC_AUTH_USER'), os.environ.get('ELASTIC_AUTH_PASSWORD')]
dataset = os.environ.get("dataset", "asb")
conversational_index = f"{dataset}-conversational"

In [ ]:
class ConversationalBatch(BaseModel):
    session_id: str
    conversational_memory: float
    conversational_insight: str
    conversational_language: float
    conversational_quality_maxim: float
    conversational_quantity_maxim: float
    conversational_relation_maxim: float
    conversational_manner_maxim: float
    conversational_sensibleness: float
    conversational_thinkings: str
    qa_id: str
    assistant_id: str

class ConversationalMetric(BaseModel):
    session_id: str
    conv_thread: List[ConversationalBatch] = []

In [ ]:
def recreate_index(index_name: str, mapping: dict):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [ ]:
class ConversationalAnalyzer(FairForge):
    def process(self, thread: Conversation):
        for batch in thread.conversation:
            query = batch.question
            # Get emotional distribution for ground truth and real assistant
            batch = ConversationalBatch(
                conversational_memory=0.0,
                conversational_insight="",
                conversational_language=0.0,
                conversational_quality_maxim=0.0,
                conversational_quantity_maxim=0.0,
                conversational_relation_maxim=0.0,
                conversational_manner_maxim=0.0,
                conversational_sensibleness=0.0,
                conversational_thinkings="",
                session_id=thread.session_id,
                qa_id=batch.qa_id,
                assistant_id=thread.assistant_id
            )
            self.metrics.append(batch)

In [ ]:
conversational = ConversationalAnalyzer()
metrics = conversational.pipeline()

In [ ]:
mapping_conversational = {
  "mappings": {
    "properties": {
      "session_id": {"type": "keyword"},
      "conversational_memory": {"type": "float"},
      "conversational_insight": {"type": "text"},
      "conversational_language": {"type": "float"},
      "conversational_quality_maxim": {"type": "float"},
      "conversational_quantity_maxim": {"type": "float"},
      "conversational_relation_maxim": {"type": "float"},
      "conversational_manner_maxim": {"type": "float"},
      "conversational_sensibleness": {"type": "float"},
      "conversational_thinkings": {"type": "text"},
      "assistant_id": {"type": "keyword"},
      "qa_id": {"type": "keyword"},
    }
  }
}